In [1]:
import pandas as pd
import numpy as np 
import sys
sys.path.append("../")
from taxcalc.records import Records
from taxcalc import Policy, Records, Calculator, Behavior, behavior
from taxcalc.utils import *
from numpy.testing import assert_array_almost_equal
import matplotlib
import matplotlib.pyplot as plt 
from matplotlib.transforms import BlendedGenericTransform
%matplotlib inline

In [2]:
puf = pd.read_csv("../puf.csv")
policy_base = Policy()
records_base = Records(puf)

policy_reform = Policy()
records_reform = Records(puf)

calcbase = Calculator(policy = policy_base, records = records_base)
calcreform = Calculator(policy = policy_reform, records = records_reform)

You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.


In [3]:
reform = {
          2015:{
        "_ID_InterestPaid_HC":[0.5],
        "_ID_StateLocalTax_HC":[0.5],
        "_ID_Charity_HC":[0.5]
    }
}
policy_reform.implement_reform(reform)

In [4]:
calcbase.advance_to_year(2015)
calcreform.advance_to_year(2015)

In [5]:
calcbase.calc_all()
calcreform.calc_all()

In [6]:
import copy

RES_COLUMNS = STATS_COLUMNS + ['e00200'] + ['MARS'] + ['n24']
# The results function selects the data frame we'll be using 
def results(c):
    outputs = []
    for col in RES_COLUMNS:
        if hasattr(c.policy, col):
            outputs.append(getattr(c.policy, col))
        else:
            outputs.append(getattr(c.records, col))
    return DataFrame(data=np.column_stack(outputs), columns=RES_COLUMNS)

# EPSILON is defined here to avoid division by zero 
EPSILON = 1e-3

# Calculating the weighted average MTR w.r.t wage
def wage_weighted(agg, col_name):
    return (float((agg[col_name] * agg['s006'] * agg['e00200']).sum())/
            ((agg['s006']*agg['e00200']).sum() + EPSILON))

# Calculating the weighted average MTR
def weighted(agg, col_name):
    return (float((agg[col_name] * agg['s006']).sum())/
            ((agg['s006']).sum() + EPSILON))

def add_income_bins(df, num_bins, tab):
    # First, sort by income_measure
    df.sort(tab, inplace=True)
    # Next, do a cumulative sum by the weights
    df['cumsum_weights'] = np.cumsum(df['s006'].values)
    # Max value of cum sum of weights
    max_ = df['cumsum_weights'].values[-1]
    # Create 100 bins and labels based on this cumulative weight
    bin_edges = [0] + list(np.arange(1, (num_bins+1)) * (max_ / float(num_bins)))
    labels = range(1, (num_bins+1))
    #  Groupby weighted deciles
    df['bins'] = pd.cut(df['cumsum_weights'], bins=bin_edges, labels=labels)
    return df

def print_data(calcX, calcY, weights, tab):
    df_x = results(calcX)
    df_y = results(calcY)
    
    id_itemizers_x = ((calcX.records.c04470 > 0) & (calcX.records.c00100 > 0))
    id_itemizers_y = ((calcY.records.c04470 > 0) & (calcY.records.c00100 > 0))
    df_x['pct_itm'] = id_itemizers_x
    df_y['pct_itm'] = id_itemizers_y
    
    
    df_y[tab] = df_x[tab]
    
    df_x = add_income_bins(df_x, 100, tab)
    df_y = add_income_bins(df_y, 100, tab)  
    
    df_filtered_x = df_x.copy()
    df_filtered_y = df_y.copy()
    
    gp_x = df_filtered_x.groupby('bins', as_index=False)
    gp_y = df_filtered_y.groupby('bins', as_index=False)
    wgtpct_x = gp_x.apply(weights, 'pct_itm')
    wgtpct_y = gp_y.apply(weights, 'pct_itm')
    #Add the bin labels
    wpct_x = DataFrame( data=wgtpct_x, columns=['w_pct'])
    wpct_y = DataFrame( data=wgtpct_y, columns=['w_pct'])
    
    wpct_x['bins'] = np.arange(1, 101)
    wpct_y['bins'] = np.arange(1, 101)
    #Join df_x and appld on the bin, carrying along 'w_mtr'
    #Left join means that 'rslt' is of size len(df_filtered_x)
    rsltx = pd.merge(df_filtered_x[['bins']], wpct_x, how='left')
    rslty = pd.merge(df_filtered_y[['bins']], wpct_y, how='left')

    # Put that column in df_filtered_x, disregarding index of rslt
    df_filtered_x['w_pct'] = rsltx['w_pct'].values
    df_filtered_y['w_pct'] = rslty['w_pct'].values
    
    df_filtered_x.drop_duplicates(subset = 'bins', inplace = True)
    df_filtered_y.drop_duplicates(subset = 'bins', inplace = True)
    
    df_filtered_x.to_csv('base.csv',float_format = '%1.3f',sep=',',header = True, index =False)
    df_filtered_y.to_csv('reform.csv',float_format = '%1.3f',sep=',',header = True, index =False)

In [7]:
print_data(calcbase, calcreform, weights = weighted, tab = 'c00100')

/Users/seanwang/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:29: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
